In [193]:
from typing import List
import sys
sys.setrecursionlimit(10**6)

class Solution:

    def helper(self, stones: List[int], turn: bool) -> int:
        n = len(stones)
        if len(stones) <= 1:
            return 0
        if turn:
            # print("Alice turn:")
            # print("stones:", stones)
            res = float('-inf')
            val = stones[0]
            for i in range(2, len(stones)+1):
                val += stones[i-1]
                new_stones = stones[i:].copy()
                new_stones.insert(0, val)
                # print("  val:", val, "new_stones:", new_stones)
                res = max(res, val + self.helper(new_stones, not turn))
            return res
        else:
            # print("    Bob turn:")
            # print("    stones:", stones)
            res = float('inf')
            val = stones[0]
            for i in range(2, len(stones)+1):
                val += stones[i-1]
                new_stones = stones[i:].copy()
                new_stones.insert(0, val)
                # print("      val:", val, "new_stones:", new_stones)
                res = min(res, -val + self.helper(new_stones, not turn))
            return res
            

    # memoized version, about O(n^2) time complexity due to the
    # fact that we call the helper function starting from the
    # smallest possible gather. I think that if we change it
    # to using the largest possible gather, we can reduce the
    # time complexity a fair bit.
    def stoneGameVIIIMemo(self, stones: List[int]) -> int:
        n = len(stones)
        memo = {}
        def gameHelper(turn: bool, i: int) -> int:
            if i >= n-1:
                return 0
            if (turn, i, stones[i]) in memo:
                return memo[(turn, i, stones[i])]

            if turn:
                # print("Alice turn:")
                # print("stones:", stones[i:])
                res = float('-inf')
                val = stones[i]
                for j in range(i+1, n):
                    old = stones[j]
                    val += old
                    stones[j] = val
                    # print("  val:", val, "new_stones:", stones[j:])
                    res = max(res, val + gameHelper(not turn, j))
                    stones[j] = old

                memo[(turn, i, stones[i])] = res
                return res
            
            else:
                # print("    Bob turn:")
                # print("    stones:", stones[i:])
                res = float('inf')
                val = stones[i]
                for j in range(i+1, n):
                    old = stones[j]
                    val += old
                    stones[j] = val
                    # print("      val:", val, "new_stones:", stones[j:])
                    res = min(res, -val + gameHelper(not turn, j))
                    stones[j] = old
                
                memo[(turn, i, stones[i])] = res
                return res
                    

        score = gameHelper(True, 0)
        return score    
    
    def stoneGameVIIIMemo2(self, stones: List[int]) -> int:
        n = len(stones)
        memo = {}
        for i in range(1, n):
            stones[i] += stones[i-1]
        
        def recursive(i: int, prefix: List[int], turn: bool) -> int:
            if i == (len(prefix) - 1):
                return prefix[i] if turn else -prefix[i]
            if (i, turn) in memo:
                return memo[(i, turn)]
            
            if turn:
                # Alice's turn
                # we can either take the current prefix and switch turn to Bob or go to the next
                # prefix and keep the turn with Alice
                # Alice is trying to maximize the score
                memo[(i, turn)] = max(recursive(i+1, prefix, turn), prefix[i] + recursive(i+1, prefix, not turn))
            else:
                # Bob's turn
                # we can either take the current prefix and switch turn to Alice or go to the next
                # prefix and keep the turn with Bob
                # Bob is trying to minimize the score
                memo[(i, turn)] = min(recursive(i+1, prefix, turn), -prefix[i] + recursive(i+1, prefix, not turn))
            return memo[(i, turn)]

        return recursive(1, stones, True)
    
    def stoneGameVIIIDP(self, stones: List[int]) -> int:
        n = len(stones)
        for i in range(1, n):
            stones[i] += stones[i-1]

        dp = [0] * n
        dp[-1] = stones[-1]
        for i in range(n-2, -1, -1):
            # Alice is trying to maximize the score difference: score[i] - dp[i+1] <= switch turn to Bob
            # Bob is trying to minimize the score difference : dp[i+1]
            dp[i] = max(stones[i] - dp[i+1], dp[i+1])

        return dp[1]

In [194]:
soln = Solution()
f = soln.stoneGameVIIIDP

# test 1
stones = [-1,2,-3,4,-5]
expected = 5
print("result:", f(stones), "expected:", expected)
print()

# test 2
stones = [7,-6,5,10,5,-2,-6]
expected = 13
print("result:", f(stones), "expected:", expected)
print()

# test 3
stones = [-10,-12]
expected = -22
print("result:", f(stones), "expected:", expected)
print()

import random
import sys
import time

sys.setrecursionlimit(10**6)
random.seed(0)

n = 10
stones = [random.randint(-100, 100) for _ in range(n)]
start = time.time()
expected = 109
print("result:", f(stones), "expected:", expected)
print("elapsed time:", time.time()-start)
print()

n = 100
stones = [random.randint(-100, 100) for _ in range(n)]
start = time.time()
expected = 221
print("result:", f(stones), "expected:", expected)
print("elapsed time:", time.time()-start)
print()

n = 1000
stones = [random.randint(-100, 100) for _ in range(n)]
start = time.time()
expected = 229
print("result:", f(stones), "expected:", expected)
print("elapsed time:", time.time()-start)
print()

n = 10000
stones = [random.randint(-100, 100) for _ in range(n)]
start = time.time()
expected = 1956
print("result:", f(stones), "expected:", expected)
print("elapsed time:", time.time()-start)
print()


result: 5 expected: 5

result: 13 expected: 13

result: -22 expected: -22

result: 109 expected: 109
elapsed time: 7.772445678710938e-05

result: 221 expected: 221
elapsed time: 0.00013184547424316406

result: 229 expected: 229
elapsed time: 0.00043201446533203125

result: 1956 expected: 1956
elapsed time: 0.003155946731567383

